In [ ]:
import sys
import os
import pandas as pd
import warnings
import json
from devtools import pprint

# Add the src folder to the path
sys.path.append(os.path.abspath('../..'))
warnings.filterwarnings("ignore", category=FutureWarning)

# Import the classes
from src.data_tool.data_providers.polygon import polygon, prepare_news_for_llm, parse_news_by_date
from src.data_tool.data_models import NewsSentiment, NewsSentimentByDate
from src.database_tool.connect_db import ConnectDB
from src.llm.prompt import prompts as p
from src.llm.api_call import call_llm
from langchain_core.prompts import ChatPromptTemplate

DB_USER and DB_PASSWORD are set


In [2]:
pl = polygon()

POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set


In [3]:
ticker = 'NVDA'
start_date = '2024-07-19'
end_date = '2024-08-06'

In [4]:
nvdia_news, nvdia_news_df = pl.get_news(ticker, start_date, end_date, limit=1000, strict=False)

Error inserting company news data for NVDA: (pymysql.err.InternalError) (1, "Can't create/write to file '/tmp/MLWb1Tx1' (OS errno 28 - No space left on device)")
[SQL: INSERT INTO company_news (polygon_id, ticker, title, author, publisher, published_utc, article_url, tickers, description, keywords, insights) VALUES (%(polygon_id)s, %(ticker)s, %(title)s, %(author)s, %(publisher)s, %(published_utc)s, %(article_url)s, %(tickers)s, %(description)s, %(keywords)s, %(insights)s)]
[parameters: [{'polygon_id': '8297fe5d499cff248a11cf4bafac6f77240ea92e303ef53cdf53ad7c52833859', 'ticker': 'NVDA', 'title': "The CEO of Nvidia just sold 700,000 shares of his company's stock. Here's what investors need to know - The Motley Fool Australia", 'author': 'Johnny Rice', 'publisher': 'The Motley Fool', 'published_utc': '2024-07-19 00:10:42', 'article_url': 'https://www.fool.com.au/2024/07/19/the-ceo-of-nvidia-just-sold-700000-shares-of-his-companys-stock-heres-what-investors-need-to-know-usfeed/', 'tickers

In [5]:
nvdia_news_df.sort_index(ascending=False)

,article_url,author,description,polygon_id,insights,keywords,published_utc,tickers,title,publisher,ticker
177,https://www.fool.com/investing/2024/08/06/opin...,The Motley Fool,The article discusses Taiwan Semiconductor Man...,6c94364db9626b7b541c480a0b37c0d3437ffe863588d7...,"[{""ticker"": ""TSM"", ""sentiment"": ""positive"", ""s...","[""Taiwan Semiconductor Manufacturing"", ""AI"", ""...",2024-08-06 08:44:00,"[""TSM"", ""NVDA"", ""MSFT""]",Opinion: This Is the Best Artificial Intellige...,The Motley Fool,NVDA
176,https://www.investing.com/analysis/this-could-...,The Tokenist,The article discusses the current market corre...,563f31761eae59974deef3b9c98a18ab136e03311ea1c7...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""AI stocks"", ""market correction"", ""Nvidia"", ""...",2024-08-06 06:29:00,"[""NVDA"", ""INTC"", ""TSM""]",This Could Be the Perfect Time to Double Down ...,Investing.com,NVDA
175,https://www.fool.com/investing/2024/08/05/this...,The Motley Fool,"Nvidia, the dominant player in the AI chip mar...",7901d22437ab0cfdf75fbde7ac82834ff704ea38243cf7...,"[{""ticker"": ""NVDA"", ""sentiment"": ""neutral"", ""s...","[""Nvidia"", ""AI"", ""Tesla"", ""Amazon"", ""Meta"", ""D...",2024-08-05 22:07:00,"[""NVDA"", ""TSLA"", ""AMZN"", ""META""]",This Artificial Intelligence (AI) Company Migh...,The Motley Fool,NVDA
174,https://www.fool.com/investing/2024/08/05/why-...,The Motley Fool,Nvidia's design flaw in its next-generation Bl...,8e8904d668ac2aa5655ef1b46cdcd5e3930a477386d942...,"[{""ticker"": ""PLTR"", ""sentiment"": ""positive"", ""...","[""Palantir Technologies"", ""Super Micro Compute...",2024-08-05 20:14:57,"[""PLTR"", ""SMCI"", ""AVGO"", ""NVDA""]","Why Palantir Technologies, Super Micro Compute...",The Motley Fool,NVDA
173,https://www.investing.com/analysis/can-tech-st...,Valuewalk,"Tech stocks, as measured by the Nasdaq 100, we...",5cdbd7d6d05268abc0b061364bc3d14ec0bfaef8291382...,"[{""ticker"": ""AAPL"", ""sentiment"": ""positive"", ""...","[""tech stocks"", ""small caps"", ""great rotation""...",2024-08-05 20:10:00,"[""AAPL"", ""GOOG"", ""GOOGL"", ""MSFT"", ""META"", ""NVD...",Can Tech Stocks Rebound in August?,Investing.com,NVDA
...,...,...,...,...,...,...,...,...,...,...,...
4,https://www.fool.com/investing/2024/07/19/is-t...,Adria Cimino,"Nvidia has become heavily dependent on AI, wit...",d9a2282f3f10df2bf321fa29e7202b9707d2737295451f...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""Nvidia"", ""AI"", ""revenue"", ""risk"", ""opportuni...",2024-07-19 08:15:00,"[""NVDA"", ""AMZN""]",Is This One Number From Nvidia a Red Flag for ...,The Motley Fool,NVDA
3,https://www.benzinga.com/markets/equities/24/0...,"Benzinga Neuro, Benzinga Staff Writer",A leading market expert warns that the 'Magnif...,3c07ea7afde2f3d9b72ffd477d702f328bbdfed2065c09...,"[{""ticker"": ""NVDA"", ""sentiment"": ""negative"", ""...","[""Nvidia"", ""Apple"", ""Microsoft"", ""Magnificent ...",2024-07-19 07:22:54,"[""NVDA"", ""AAPL"", ""MSFT""]","Nvidia, Apple, And Microsoft May Face Potentia...",Benzinga,NVDA
2,https://uk.investing.com/analysis/tsla-enters-...,Adam Harris,Tesla (TSLA) has seen a decline in its market ...,b85640ae3f171b4b8eca4437601afaeea18cbe11434c11...,"[{""ticker"": ""TSLA"", ""sentiment"": ""negative"", ""...","[""Tesla"", ""electric vehicles"", ""competition"", ...",2024-07-19 06:45:00,"[""TSLA"", ""GM"", ""NVDA"", ""VWAGY""]",TSLA enters wedge- will it go back down to $21...,Investing.com,NVDA
1,https://www.fool.com.au/2024/07/19/should-we-b...,Bernd Struben,The tech-heavy Nasdaq index has seen a 3.4% de...,117c7176d39fa977dcc2c0580fe435b93f8b6441bc0c3b...,"[{""ticker"": ""NVDA"", ""sentiment"": ""negative"", ""...","[""Nasdaq"", ""tech stocks"", ""market crash"", ""sto...",2024-07-19 02:49:11,"[""NVDA""]",Should we be worried about a Nasdaq stock mark...,The Motley Fool,NVDA


In [6]:
LLM_news = prepare_news_for_llm(nvdia_news.results)

In [7]:
template = p.news_sentiment_template

prompt = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker})

In [8]:
print(prompt)

messages=[SystemMessage(content='You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.\n                    You should also identify if the news mentioned any upcoming key events that could impact the stock price.\n                ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Based on the following news, create the investment signal:\n                    News Data for NVDA\n                    [\n  {\n    "tickers": [\n      "NVDA",\n      "AMD"\n    ],\n    "title": "The CEO of Nvidia just sold 700,000 shares of his company\'s stock. Here\'s what investors need to know - The Motley Fool Australia",\n    "description": "The CEO of Nvidia, Jensen Huang, sold 700,000 shares of the company\'s stock, which amounts to less than 0.1% of his net worth. While this may raise questions, experts advise investors to focus on evaluating the overall busines

In [9]:
model_name = "gemini-2.0-flash"
model_provider = "Gemini"
pydantic_model = NewsSentiment
max_retries = 3

In [10]:
sentiment_result = call_llm(prompt, model_name, model_provider, pydantic_model, max_retries=max_retries)

In [11]:
pprint(sentiment_result)

NewsSentiment(
    short_term_sentiment=ShortTermSentiment(
        short_term_sentiment='Neutral',
        short_term_confidence=0.6,
        short_term_reasoning="Nvidia's stock price has been volatile recently, but the company's fundamentals remain strong.",
    ),
    long_term_sentiment=LongTermSentiment(
        long_term_sentiment='Positive',
        long_term_confidence=0.7,
        long_term_reasoning='Nvidia is well-positioned to benefit from the growth of AI.',
    ),
    key_events_outlook=[],
)


In [12]:
print(type(sentiment_result))

<class 'src.data_tool.data_models.NewsSentiment'>


In [13]:
template = p.news_sentiment_by_date_template


In [14]:
sentiment_results_by_date = []
for date, news in parse_news_by_date(nvdia_news.results).items():
    llm_news = prepare_news_for_llm(news)
    prompt_news_by_date = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker, "date": date} )
    print(f'\nProcessing news on {date}: {len(news)} news')
    sentiment_result_by_date = call_llm(prompt_news_by_date, model_name, model_provider, NewsSentimentByDate, max_retries=max_retries)
    sentiment_results_by_date.append(sentiment_result_by_date)


17 days of news parsed

Processing news on 2024-07-19: 20 news

Processing news on 2024-07-20: 11 news

Processing news on 2024-07-21: 9 news

Processing news on 2024-07-22: 20 news

Processing news on 2024-07-23: 17 news

Processing news on 2024-07-24: 15 news

Processing news on 2024-07-26: 10 news

Processing news on 2024-07-28: 3 news

Processing news on 2024-07-29: 9 news

Processing news on 2024-07-30: 13 news

Processing news on 2024-07-31: 10 news

Processing news on 2024-08-01: 7 news

Processing news on 2024-08-02: 9 news

Processing news on 2024-08-03: 5 news

Processing news on 2024-08-04: 6 news


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 59
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 57
}
].
Retrying langchain_google_gena

Error in LLM call after 3 attempts: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 52
}
]

Processing news on 2024-08-05: 12 news

Processing news on 2024-08-06: 2 news


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 50
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 47
}
].
Retrying langchain_google_gena

In [15]:
len(sentiment_results_by_date)

17

In [16]:
for result in sentiment_results_by_date:
    pprint(result)

NewsSentimentByDate(
    summary=(
        'News about NVDA on 2024-07-19 is mixed. It includes positive developments such as strong AI chip demand, Blac'
        'kwell architecture benefits, and investments in robotics. However, there are negative aspects like CEO inside'
        'r selling, market correction concerns, competition, AI bubble risks, and geopolitical tensions.'
    ),
    sentiment='mixed',
    reasoning=(
        'The news articles present a mixed sentiment. There are positive articles related to AI chip demand and potent'
        "ial benefits from the Blackwell architecture, as well as Nvidia's investments in Serve Robotics. However, the"
        're are also negative articles discussing insider selling by the CEO, potential market correction, competition'
        ' from other companies, and concerns about the AI bubble bursting. The potential for export restrictions and g'
        'eopolitical tensions also contributes to the mixed sentiment.'
    ),
    date='2024